# RPA trazer informações do banco do 2o para o banco do 1o

In [ ]:
import os
import psycopg2
from pymongo import MongoClient
import pandas as pd
from dotenv import load_dotenv

In [ ]:
load_dotenv()

host_1ano = os.getenv("DB_HOST_1ano")
port_1ano = int(os.getenv("DB_PORT_1ano"))
user_1ano = os.getenv("DB_USER_1ano")
password_1ano = os.getenv("DB_PASSWORD_1ano")
database_1ano = os.getenv("DB_NAME_1ano")
db_url_1ano = os.getenv("DATABASE_URL_1ano")

host_2ano = os.getenv("DB_HOST_2ano")
port_2ano = int(os.getenv("DB_PORT_2ano"))
user_2ano = os.getenv("DB_USER_2ano")
password_2ano = os.getenv("DB_PASSWORD_2ano")
database_2ano = os.getenv("DB_NAME_2ano")
db_url_2ano = os.getenv("DATABASE_URL_2ano")


conexao = MongoClient(os.getenv("DB_MONGO_URL"))
db_name = conexao[os.getenv("DB_MONGO_NAME")]
collection_funcoes = db_name[os.getenv("DB_MONGO_COLLECTION_FUNCOES")] 
collection_situacao_trabalhista = db_name[os.getenv("DB_MONGO_COLLECTION_SITUACAO_TRABALHISTA")]

## tipo_vaga

In [ ]:
try:

    # conectando-se ao banco do 1ano
    conn_bd1 = psycopg2.connect(
        user=user_1ano,
        host=host_1ano,
        port=port_1ano,
        password=password_1ano,
        database=database_1ano,
    )

    # conectando-se ao banco do 2ano
    conn_bd2 = psycopg2.connect(
        user=user_2ano,
        host=host_2ano,
        port=port_2ano,
        password=password_2ano,
        database=database_2ano,
    )

    # Lendo os valores da tabela tip_vaga do 2ano
    df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM tipo_vaga;", conn_bd2)

    # Criando um cursor para poder fazer queries no banco do 1ano
    cursor_bd1 = conn_bd1.cursor()

    for i in range(len(df_tipo_vaga_bd2)):
        # Lendo os valores de cada linha do dataframe que contém as informações da tabela tipo_vaga do 2ano
        values = (df_tipo_vaga_bd2['id'][i], df_tipo_vaga_bd2['nome'][i])

        # Query para atualizar a tabela do 1o com as informações do 2o
        update_query = "UPDATE public.tipo_vaga SET nome = %s WHERE id = %s;"
        
        try:
            # Executando a query
            cursor_bd1.execute(update_query, values)

            # Se a query de atualização não tiver nenhuma linha alterada, então significa que os valores na tabela do 2o são novos
            if cursor_bd1.rowcount == 0:
                # query para inserir
                insert_query = "INSERT INTO public.tipo_vaga(id, nome) VALUES (%s, %s);"
                # executando a query de insert
                cursor_bd1.execute(insert_query, values)

            conn_bd2.commit()

            conn_bd2.close()
            conn_bd1.close()

        except Exception as e:
            print(f"Erro ao executar a query para id {values[0]}: {e}")

except Exception as e:
    print("Error connecting to PostgreSQL:", e)

